In [ ]:
## Install necessary packages in terminal using pip install (cartopy, geopandas)
import xarray as xr
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import cartopy.feature as cfeature
from cartopy.io.shapereader import Reader
from cartopy.feature import ShapelyFeature
import cartopy.io.shapereader as shpreader
import cmocean.cm as cmo
import numpy as np
from cartopy.mpl.gridliner import LONGITUDE_FORMATTER, LATITUDE_FORMATTER
import pandas as pd
import matplotlib.ticker as ticker
import seawater
import matplotlib.gridspec as gridspec
import geopandas as gpd
from matplotlib.offsetbox import (TextArea, DrawingArea, OffsetImage,
                                  AnnotationBbox)
import matplotlib.ticker as mticker
from matplotlib.lines import Line2D
from scipy.ndimage.filters import gaussian_filter
from cartopy.feature import NaturalEarthFeature, LAND, COASTLINE, RIVERS


In [ ]:

#import cmocean
hfont = {'fontname':'Helvetica'}

## Read in you excel sheet with your lat lons
df = pd.read_excel('/Users/bhorwitz/Dropbox (Personal)/MTS/njsites.xls') ## change this path

In [ ]:
## Plot the map
plt.figure(figsize=(22,36)) ## Figure size (matplotlib) 


ax = plt.axes(projection=ccrs.PlateCarree()) ## this is the map projecttion you can just leave this


fname  = 'https://www.ngdc.noaa.gov/thredds/dodsC/global/ETOPO2022/30s/30s_bed_elev_netcdf/ETOPO_2022_v1_30s_N90W180_bed.nc'

ds_etpo = xr.open_dataset(fname) 
lon1 = -75 ## These are the map extents. Right now it's set to include NJ but you can change this
lon2 = -73.2
lat1 = 38.5
lat2 = 40
etopo1= ds_etpo.sel(lon=slice(lon1, lon2), lat=slice(lat1, lat2))
bathy_levels = [-45,-30,-15]  ## These are the bathymetry levels you are pulling

dpth_contours = np.arange(-50, 1, 1)

ax.contour(etopo1.lon, etopo1.lat, etopo1.z, bathy_levels, transform=ccrs.PlateCarree(), colors='black',latlon=True,
linewidths=0.7,linestyles='solid', zorder=9) ##This is plotting the bathymetry levels you can change these (this is just matplotlib settings) 



ax.add_feature(cfeature.LAND, color="lightgrey", alpha=0.3) ##You can change the color or the translucency here
ax.add_feature(cfeature.OCEAN, color="skyblue", alpha=0.2)
shapename = 'admin_1_states_provinces_lakes'
states_shp = shpreader.natural_earth(resolution='110m',
                                         category='cultural', name=shapename) ##This is adding states. If you want to take the states out just delete this
ax.add_geometries(
        shpreader.Reader(states_shp).geometries(),
        ccrs.PlateCarree(),edgecolor="black", facecolor="lightgrey", alpha=0.3) ##This is adding states. If you want to take the states out just delete this
## These are just putting the lat lons on the figure
gridliner = ax.gridlines(crs=ccrs.PlateCarree(), draw_labels=True)
gridliner.xlabels_top = False
gridliner.xlabels_bottom = True
gridliner.ylabels_left = True
gridliner.ylabels_right = False
gridliner.ylines = False  # you need False
gridliner.xlines = False
gridliner.xlabel_style = {'size': 20, 'color': 'black', **hfont,'weight': 'bold'}
gridliner.ylabel_style = {'color': 'black', 'size':20, **hfont,'weight': 'bold' }


ax.scatter(x=df["Lon"],y=df["Lat"],edgecolor='black',c=df["cluster"],s=350) ##This is where you add in your lat lons for the scatter points. make sure your column names match 
#f._kwargs['edgecolor']='black'


ax.coastlines(resolution='10m', linewidth=1.5)

ax.set_extent([-75.5,-72.5,38,41]) #this is also the extent of the map make sure you match this to the top ones

plt.savefig('/Users/bhorwitz/Dropbox (Personal)/MTS/Figures/Map3.png',dpi=400); #this is how you save the figure to your computer (make sure this path is your own) 